# Домашнее задание 6. Оценка LLM на классификационных задачах.

Перед выполнением задания рекомендуем ознакомиться с этой [статьей](https://huggingface.co/blog/open-llm-leaderboard-mmlu).

В этом задании вы будете работать с моделью `Qwen/Qwen2-0.5B` и частью датасета `cais/mmlu` - "medical_genetics".

In [111]:
# При необходимости установим нужные зависимости
%pip -q install torch transformers datasets matplotlib

In [112]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

In [114]:
model_name = "Qwen/Qwen2-0.5B"  # Можно заменить на другую модель
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

## Загрузка датасета

Загрузим наши данные. В датасете есть три части:
* test
* validation
* dev

Мы с вами будем использовать **test** для расчета метрик и **dev** для примеров в случае few-shot валидации.

In [115]:
mmlu = load_dataset("cais/mmlu", "medical_genetics")

mmlu

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 11
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 5
    })
})

Выведем несколько примеров из датасета:

In [116]:
# Пример сэмпла из датасета
print("Примеры из датасета:")
for i in range(2):
    sample = mmlu['dev'][i]
    print(f"Subject: {sample['subject']}")
    print(f"Question: {sample['question']}")
    print(f"Choices: {sample['choices']}")
    print(f"Answer: {sample['answer']}\n")

Примеры из датасета:
Subject: medical_genetics
Question: Large triplet repeat expansions can be detected by:
Choices: ['polymerase chain reaction.', 'single strand conformational polymorphism analysis.', 'Southern blotting.', 'Western blotting.']
Answer: 2

Subject: medical_genetics
Question: DNA ligase is
Choices: ['an enzyme that joins fragments in normal DNA replication', 'an enzyme of bacterial origin which cuts DNA at defined base sequences', 'an enzyme that facilitates transcription of specific genes', 'an enzyme which limits the level to which a particular nutrient reaches']
Answer: 0



## Вспомогательные функции

In [117]:
def get_model_logits(inputs: dict[str, torch.Tensor], model: AutoModelForCausalLM) -> torch.Tensor:
    """
    Make forward pass for the model.

    Args:
        inputs dict(str, torch.Tensor): The inputs tensors after tokenization.
        model (AutoModelForCausalLM): The language model used to generate logits.

    Returns:
        torch.Tensor: The logits output from the model.
    """

    # Получение логитов модели
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return logits

# Имплементация функций для решения ДЗ

### Сборка промптов


Ваша задача написать реализацию в файле `collect_prompt.py`:


**0-shot prompt. Функция `create_prompt(sample: dict) -> str:`**

* Эта функция должна принимать на вход словарь sample, содержащий информацию о вопросе, предмете, вариантах ответов и индексе правильного ответа.
* На выходе функция должна возвращать промпт по следующему формату:

```
The following are multiple choice questions (with answers) about {subject}.
{question}
A. {option_0}
B. {option_1}
C. {option_2}
D. {option_3}
Answer:
```

**Пример результата функции:**

```
The following are multiple choice questions (with answers) about medical_genetics.
In a Robertsonian translocation fusion occurs at the:
A. telomeres.
B. centromeres.
C. histones.
D. ends of the long arms.
Answer:
```


**5-shot prompt. Функция `create_prompt_with_examples(sample: dict, examples: list, add_full_example: bool = False) -> str:`**

* Эта функция должна принимать на вход словарь sample, список из 5 примеров examples из части dev датасета и булевый параметр
add_full_example.
* Если параметр add_full_example установлен в True, то в каждом примере должно быть указано полное содержание правильного ответа. Это необходимо для двух вариантов валидации, которые будут рассмотрены далее
* На выходе функция должна возвращать строку по следующему формату если `add_full_example=False`:

```
The following are multiple choice questions (with answers) about {subject}.
{question}
A. {option_0}
B. {option_1}
C. {option_2}
D. {option_3}
Answer: {letter_of_correct_option}

<other examples...>

The following are multiple choice questions (with answers) about {subject}.
{question}
A. {option_0}
B. {option_1}
C. {option_2}
D. {option_3}
Answer:
```

**Пример результата функции:**

```
The following are multiple choice questions (with answers) about medical_genetics.
Large triplet repeat expansions can be detected by:
A. polymerase chain reaction.
B. single strand conformational polymorphism analysis.
C. Southern blotting.
D. Western blotting.
Answer: C

The following are multiple choice questions (with answers) about medical_genetics.
DNA ligase is
A. an enzyme that joins fragments in normal DNA replication
B. an enzyme of bacterial origin which cuts DNA at defined base sequences
C. an enzyme that facilitates transcription of specific genes
D. an enzyme which limits the level to which a particular nutrient reaches
Answer: A

The following are multiple choice questions (with answers) about medical_genetics.
A gene showing codominance
A. has both alleles independently expressed in the heterozygote
B. has one allele dominant to the other
C. has alleles tightly linked on the same chromosome
D. has alleles expressed at the same time in development
Answer: A

The following are multiple choice questions (with answers) about medical_genetics.
Which of the following conditions does not show multifactorial inheritance?
A. Pyloric stenosis
B. Schizophrenia
C. Spina bifida (neural tube defects)
D. Marfan syndrome
Answer: D

The following are multiple choice questions (with answers) about medical_genetics.
The stage of meiosis in which chromosomes pair and cross over is:
A. prophase I
B. metaphase I
C. prophase II
D. metaphase II
Answer: A

The following are multiple choice questions (with answers) about medical_genetics.
In a Robertsonian translocation fusion occurs at the:
A. telomeres.
B. centromeres.
C. histones.
D. ends of the long arms.
Answer:
```


* На выходе функция должна возвращать строку по следующему формату если `add_full_example=True`:

```
The following are multiple choice questions (with answers) about {subject}.
{question}
A. {option_0}
B. {option_1}
C. {option_2}
D. {option_3}
Answer: {letter_of_correct_option}. {correct_option}

<other examples...>

The following are multiple choice questions (with answers) about {subject}.
{question}
A. {option_0}
B. {option_1}
C. {option_2}
D. {option_3}
Answer:
```

Единственное отличие в этом случае в том, что после `Answer` вставляется не одна буква, а весь вариант ответа.

**Пример результата функции:**

```
The following are multiple choice questions (with answers) about medical_genetics.
Large triplet repeat expansions can be detected by:
A. polymerase chain reaction.
B. single strand conformational polymorphism analysis.
C. Southern blotting.
D. Western blotting.
Answer: C. Southern blotting.

The following are multiple choice questions (with answers) about medical_genetics.
DNA ligase is
A. an enzyme that joins fragments in normal DNA replication
B. an enzyme of bacterial origin which cuts DNA at defined base sequences
C. an enzyme that facilitates transcription of specific genes
D. an enzyme which limits the level to which a particular nutrient reaches
Answer: A. an enzyme that joins fragments in normal DNA replication

The following are multiple choice questions (with answers) about medical_genetics.
A gene showing codominance
A. has both alleles independently expressed in the heterozygote
B. has one allele dominant to the other
C. has alleles tightly linked on the same chromosome
D. has alleles expressed at the same time in development
Answer: A. has both alleles independently expressed in the heterozygote

The following are multiple choice questions (with answers) about medical_genetics.
Which of the following conditions does not show multifactorial inheritance?
A. Pyloric stenosis
B. Schizophrenia
C. Spina bifida (neural tube defects)
D. Marfan syndrome
Answer: D. Marfan syndrome

The following are multiple choice questions (with answers) about medical_genetics.
The stage of meiosis in which chromosomes pair and cross over is:
A. prophase I
B. metaphase I
C. prophase II
D. metaphase II
Answer: A. prophase I

The following are multiple choice questions (with answers) about medical_genetics.
In a Robertsonian translocation fusion occurs at the:
A. telomeres.
B. centromeres.
C. histones.
D. ends of the long arms.
Answer:
```

### Получение предсказаний

Ваша задача написать реализацию в файле `get_predictions.py`:

**Token ID. Функция `predict_by_token_id(logits: torch.Tensor, tokenizer: AutoTokenizer) -> int:`**

* Функция определяет предсказанный выбор на основе логитов, полученных от модели.
* Параметры:
  * `logits` — логиты, полученные от модели, обычно имеют размерность `(1, sequence_length, vocab_size)`. Эти логиты представляют собой необработанные предсказания модели до применения любой функции активации, такой как softmax.
  * `tokenizer` — токенизатор, используемый для кодирования входной последовательности.

Указания:
* Получите логиты для последнего токена, предполагая, что он соответствует токену ответа.
* Используйте токенизатор для кодирования вариантов ['A', 'B', 'C', 'D'] без добавления специальных токенов.
* Определите предсказанный класс как индекс максимального значения логита.

**Continuation. Функция `get_choice_log_probs(logits: torch.Tensor, input_ids: torch.Tensor) -> float:`**

* Функция вычисляет среднее значение логарифмов вероятностей предсказанных токенов для заданной последовательности.
* Параметры:
    *  `logits` — логиты, сгенерированных моделью, размерностью (batch_size, sequence_length, vocab_size). Логиты представляют собой необработанные предсказания модели до применения функции softmax.
    *  `input_ids` содержит фактические токены, которые были поданы на вход модели, размерностью (batch_size, sequence_length)

Указания:
* Используйте функцию torch.nn.functional.log_softmax для преобразования логитов в логарифмы вероятностей.
* Учтите, что модель предсказывает следующий токен, поэтому необходимо сдвинуть input_ids и logits, чтобы правильно сопоставить их.
* Можно использовать torch.gather для извлечения логарифмов вероятностей, соответствующих фактическим токенам в input_ids, но можно воспользоваться любым другим способом

### Импорт реализованных функций

In [120]:
from get_predictions import predict_by_token_id, get_choice_log_probs
from collect_prompt import create_prompt, create_prompt_with_examples

# 0-shot валидация

### Предсказание класса по индексу соответствующего варианта

In [121]:
predictions = []
correct_answers = [sample['answer'] for sample in mmlu['test']]

for sample in tqdm(mmlu['test'], desc="Processing prompts"):
    prompt = create_prompt(sample)
    inputs = tokenizer(prompt, return_tensors='pt')

    # Получение логитов модели
    logits = get_model_logits(inputs, model)

    # Определение предсказания
    predicted_choice = predict_by_token_id(logits, tokenizer)
    predictions.append(predicted_choice)

Processing prompts:   0%|          | 0/100 [00:00<?, ?it/s]

In [122]:
# Расчет accuracy с использованием sklearn
accuracy_zero_shot = accuracy_score(correct_answers, predictions)
print(f"Accuracy: {accuracy_zero_shot:.3f}")

Accuracy: 0.400


### Предсказание класса по среднему лог вероятностей

In [123]:
predictions = []
correct_answers = [sample['answer'] for sample in mmlu['test']]

for sample in tqdm(mmlu['test'], desc="Processing prompts"):
    base_prompt = create_prompt(sample)
    choice_log_probs = []

    for choice, letter in zip(sample['choices'], ["A", "B", "C", "D"]):
        # Создание полного текста для каждого варианта
        full_prompt = base_prompt + f" {letter}. {choice}"
        inputs = tokenizer(full_prompt, return_tensors='pt')

        # Получение логитов модели
        logits = get_model_logits(inputs, model)

        # Получение среднего логарифмов вероятностей
        choice_log_prob = get_choice_log_probs(logits, inputs['input_ids'])
        choice_log_probs.append(choice_log_prob)

    # Определение предсказания как индекс максимального среднего логарифмов вероятностей
    predicted_choice = choice_log_probs.index(max(choice_log_probs))
    predictions.append(predicted_choice)

Processing prompts:   0%|          | 0/100 [00:00<?, ?it/s]

In [124]:
accuracy_zero_shot_continuation = accuracy_score(correct_answers, predictions)
print(f"Accuracy: {accuracy_zero_shot_continuation:.3f}")

Accuracy: 0.390


# 5-shot валидация

### Предсказание класса по индексу соответствующего варианта

In [125]:
predictions = []
correct_answers = [sample['answer'] for sample in mmlu['test']]

for sample in tqdm(mmlu['test'], desc="Processing prompts"):
    prompt = create_prompt_with_examples(sample, mmlu['dev'])
    inputs = tokenizer(prompt, return_tensors='pt')
    # Получение логитов модели
    logits = get_model_logits(inputs, model)

    # Определение предсказания
    predicted_choice = predict_by_token_id(logits, tokenizer)
    predictions.append(predicted_choice)

Processing prompts:   0%|          | 0/100 [00:00<?, ?it/s]

In [126]:
accuracy_five_shot = accuracy_score(correct_answers, predictions)
print(f"Accuracy: {accuracy_five_shot:.3f}")

Accuracy: 0.450


### Предсказание класса по среднему лог вероятностей

In [127]:
predictions = []
correct_answers = [sample['answer'] for sample in mmlu['test']]

for sample in tqdm(mmlu['test'], desc="Processing prompts"):
    base_prompt = create_prompt_with_examples(sample, mmlu['dev'], add_full_example=True)
    choice_log_probs = []

    for choice, letter in zip(sample['choices'], ["A", "B", "C", "D"]):
        # Создание полного текста для каждого варианта
        full_prompt = base_prompt + f" {letter}. {choice}"
        inputs = tokenizer(full_prompt, return_tensors='pt')

        # Получение логитов модели
        logits = get_model_logits(inputs, model)

        # Получение суммы логарифмов вероятностей
        choice_log_prob = get_choice_log_probs(logits, inputs['input_ids'])
        choice_log_probs.append(choice_log_prob)

    # Определение предсказания как индекс максимальной суммы логарифмов вероятностей
    predicted_choice = choice_log_probs.index(max(choice_log_probs))
    predictions.append(predicted_choice)

Processing prompts:   0%|          | 0/100 [00:00<?, ?it/s]

In [128]:
accuracy_five_shot_continuation = accuracy_score(correct_answers, predictions)
print(f"Accuracy: {accuracy_five_shot_continuation:.3f}")

Accuracy: 0.390


# Сравнение результатов

In [129]:
results = pd.DataFrame(
    [
        ['0-shot', accuracy_zero_shot],
        ['0-shot continuation', accuracy_zero_shot_continuation],
        ['5-shot', accuracy_five_shot],
        ['5-shot continuation', accuracy_five_shot_continuation]
    ],
    columns = ['method', 'accuracy']
)

results

,method,accuracy
0,0-shot,0.40
1,0-shot continuation,0.39
2,5-shot,0.45
3,5-shot continuation,0.39
